In [27]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [5]:
!pip list

Package                                 Version
--------------------------------------- -------------------
accelerate                              1.3.0
aiobotocore                             2.5.0
aiofiles                                22.1.0
aiohappyeyeballs                        2.4.4
aiohttp                                 3.11.10
aioitertools                            0.7.1
aiosignal                               1.2.0
aiosqlite                               0.18.0
alabaster                               0.7.16
anaconda-anon-usage                     0.4.2
anaconda-catalogs                       0.2.0
anaconda-client                         1.12.1
anaconda-project                        0.11.1
annotated-types                         0.7.0
anyio                                   4.6.2
appdirs                                 1.4.4
argon2-cffi                             21.3.0
argon2-cffi-bindings                    21.2.0
arrow                                   1.3.0
astroid  

In [28]:
import sys
print(sys.executable)
!pip install unsloth tokenizer torch unsloth_zoo -q

/home/h2o-llmstudio-onprem/anaconda3/bin/python


In [29]:
!pip show torch
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /home/h2o-llmstudio-onprem/.local/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, cut-cross-entropy, detoxify, llama-index-llms-huggingface, optimum, peft, sentence-transformers, torchaudio, torchvision, unsloth_zoo, xformers
True
1
Tesla T4


In [30]:
!pip install peft -q

In [31]:
from unsloth import FastLanguageModel
import torch
#from .autonotebook import tqdm as notebook_tqdm

max_seq_length = 2048  # Choose any max sequence length
dtype = None  # None for auto-detection. Float16 for Tesla T4/V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4-bit quantization to reduce memory usage. Can be False.

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-v0.2-bnb-4bit",  # Choose from available UnsLoft models
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)


==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
# Configure the model with LoRA and Unsloth parameters for optimized fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Suggested values: 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=16,
    lora_dropout=0,  # Optimized with 0 dropout
    bias="none",  # Optimized with "none"
    use_gradient_checkpointing="unsloth",  # Reduces VRAM usage
    random_state=3407,
    use_rslora=False,  # Supports Rank Stabilized LoRA
    loftq_config=None  # LoftQ configuration
)

print("Model configured successfully!")

Unsloth: Already have LoRA adapters! We shall skip this step.


Model configured successfully!


In [33]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

print("Model configured successfully!")

Unsloth: Already have LoRA adapters! We shall skip this step.


Model configured successfully!


In [34]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("KingNish/reasoning-base-20k", split="train")

# Rename columns (assuming 'user' -> 'instruction', 'reasoning' -> 'input', 'assistant' -> 'output')
dataset = dataset.rename_columns({"user": "instruction", "reasoning": "input", "assistant": "output"})

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    
    texts = [
        alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        for instruction, input_text, output in zip(instructions, inputs, outputs)
    ]
    
    return {"text": texts}

# Apply formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)


# Print sample data to verify
print(dataset[0]["text"])  # Check if format is correct


Map:   0%|          | 0/19944 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.

### Input:
Let's break down the problem step by step and reason through it.

**Understanding the Problem**

The problem asks us to prove that the difference between two consecutive cubes cannot be divisible by 5. To approach this problem, we need to understand what consecutive cubes are and how we can represent them mathematically.

Consecutive cubes are cubes of consecutive integers. For example, if we take the integer 2, its cube is 2^3 = 8. The next consecutive integer is 3, and its cube is 3^3 = 27. So, the difference between these two consecutive cubes is 27 - 8 = 19.

**Representing Consecutive Cubes Mathematically**

Let's represen

In [35]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
 
# Load Mistral model with Unsloth
MODEL_NAME = "unsloth/mistral-7b-v0.2-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    MODEL_NAME,
    max_seq_length=1024,  # Adjust sequence length as needed
    dtype=torch.float16,  # Use bfloat16 for better precision on some GPUs
    load_in_4bit=True,  # Set True to use 4-bit quantization for efficiency
)
 
# Tokenization
prompt = """Act as a mathematical expert and solve the problem:

Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1."""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
 
# Generate response
with torch.inference_mode():
    output = model.generate(**inputs, max_new_tokens=100)
 
# Decode output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Act as a mathematical expert and solve the problem:

Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.

Solution:

Let $x$ be a cube. Then $x^3$ is divisible by 5.

If $x^3$ is divisible by 5, then $x$ is divisible by 5.

If $x$ is divisible by 5, then $x^3$ is divisible by 5.

If $x^3$ is divisible by 5, then $x^


In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

# Load Mistral model with Unsloth
MODEL_NAME = "unsloth/mistral-7b-v0.2-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    MODEL_NAME,
    max_seq_length=2048,  # Adjust sequence length as needed
    dtype=torch.float16,  # Use bfloat16 if supported for better precision
    load_in_4bit=True,  # Use 4-bit quantization for efficiency
)

# Define and assign the chat template to the tokenizer
chat_template = """{%- for message in messages %}
{{ '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>\n' }}
{%- endfor %}"""
tokenizer.chat_template = chat_template


# LoRA Configuration for Efficient Fine-Tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Optimized for efficiency
    bias="none",
    use_gradient_checkpointing="unsloth",  # Reduces VRAM usage
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("Model configured successfully!")

# Load dataset
dataset = load_dataset("KingNish/reasoning-base-20k", split="train")

# Rename columns to match instruction format
dataset = dataset.rename_columns({"user": "instruction", "reasoning": "input", "assistant": "output"})

# Convert dataset to chat-style format
def convert_to_chat_format(example):
    chat = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["output"]},
    ]
    return {"chat": chat}

dataset = dataset.map(convert_to_chat_format)

# Apply new chat template formatting
dataset = dataset.map(lambda x: {
    "formatted_chat": tokenizer.apply_chat_template(
        x["chat"], tokenize=False, add_generation_prompt=False, add_special_tokens=False
    )
})

# Print a formatted sample
print(dataset["formatted_chat"][0])

# Training Configuration
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="formatted_chat",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # Set num_train_epochs=1 for full training runs
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use this for WandB, TensorBoard, etc.
    ),
)

print("Training setup complete!")


==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model configured successfully!


Map:   0%|          | 0/19944 [00:00<?, ? examples/s]

<|im_start|>user
Prove that the difference between two consecutive cubes cannot be divisible by 5, using the fact that the only possible remainders when a cube is divided by 5 are 0, 1, and -1.<|im_end|>
<|im_start|>assistant
Let the two consecutive cubes be $n^3$ and $(n+1)^3$. Their difference is:

$$(n+1)^3 - n^3 = 3n^2 + 3n + 1.$$

When $n^3$ is divided by 5, the possible remainders are 0, 1, and -1. Therefore, when $(n+1)^3$ is divided by 5, the possible remainders are 1, 2, and 0, respectively.

Thus, the possible remainders when the difference between the cubes is divided by 5 are:

* 1 - 0 = 1
* 2 - 1 = 1
* 0 - (-1) = 1

Since none of these remainders is 0, the difference between two consecutive cubes cannot be divisible by 5.<|im_end|>



Map (num_proc=2):   0%|          | 0/19944 [00:00<?, ? examples/s]

Training setup complete!


In [3]:
import torch

# Get GPU stats
gpu_stats = torch.cuda.get_device_properties(0)

# Get reserved and total memory
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)  # Convert to GB
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)  # Convert to GB

# Print GPU memory stats
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.581 GB.
7.885 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24,926 | Num Epochs = 1 | Total steps = 3,115
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/3,794,014,208 (1.11% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
50,0.812800
100,0.727100
150,0.700100
200,0.704400
250,0.707000
300,0.693800
350,0.717500
400,0.723000
450,0.721700
500,0.683800


In [1]:
import torch

# Calculate used memory
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)  # Convert to GB
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)  # Memory used for LoRA

# Calculate percentage usage
used_percentage = round((used_memory / max_memory) * 100, 3)
lora_percentage = round((used_memory_for_lora / max_memory) * 100, 3)

# Print training stats
trainer_stats = trainer.state.log_history[-1]  # Get latest training log
train_runtime = trainer_stats.get("train_runtime", 0)

print(f"{train_runtime} seconds used for training.")
print(f"{round(train_runtime / 60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage}%.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage}%.")


NameError: name 'start_gpu_memory' is not defined

In [2]:
model.save_pretrained("my_model1")

NameError: name 'model' is not defined

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, TextStreamer

# Define the path to your saved fine-tuned model
model_path = "my_model1"

# Load the fine-tuned model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    load_in_4bit=True  # Use 4-bit quantization for efficiency
)

# Enable faster inference
FastLanguageModel.for_inference(model)


In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch


eval_prompt = """

Act as mathematical expert and solve the problem

How many four digit numbers greater than 2999 can be formed such that the product of the middle two digits exceeds 5?
"""

# Tokenize input
inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set up text streamer for real-time output
text_streamer = TextStreamer(tokenizer)

# Generate response
model.eval()
with torch.no_grad():
    output_tokens = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.3,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

# Print the complete response
print("\n===== Insurance Underwriting Analysis =====\n")
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch


eval_prompt = """

Act as mathematical expert and solve the problem

A board game spinner is divided into three parts labeled $A$, $B$ and $C$. The probability of the spinner landing on $A$ is $\frac{1}{3}$ and the probability of the spinner landing on $B$ is $\frac{5}{12}$. What is the probability of the spinner landing on $C$? Express your answer as a common fraction.

"""

# Tokenize input
inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set up text streamer for real-time output
text_streamer = TextStreamer(tokenizer)

# Generate response
model.eval()
with torch.no_grad():
    output_tokens = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.3,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

# Print the complete response
print("\n===== Insurance Underwriting Analysis =====\n")
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer

# First verify model is properly loaded
print(f"Model loaded: {model.__class__.__name__}")
print(f"Number of parameters: {model.num_parameters()}")
print(f"Model device: {next(model.parameters()).device}")

# Enable Unsloth optimization
FastLanguageModel.for_inference(model)

eval_prompt = """Act as mathematical expert and solve the problem
How many four digit numbers greater than 2999 can be formed such that the product of the middle two digits exceeds 5?
"""

# Ensure model is in evaluation mode
model.eval()

# Create inputs
inputs = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
text_streamer = TextStreamer(tokenizer)

# Generate with appropriate parameters for a 7B parameter model
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=1024,  # Adjusted for your use case
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

print("\n===== Complete Response =====\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))